Try to use these to fix outputs issue:

https://ipython.readthedocs.io/en/stable/interactive/magics.html
https://notebook.community/lifeinoppo/littlefishlet-scode/RES/REF/python_sourcecode/ipython-master/examples/IPython%20Kernel/Capturing%20Output

# TODO: Add to references for using dgl library

@article{wang2019dgl,
    title={Deep Graph Library: A Graph-Centric, Highly-Performant Package for Graph Neural Networks},
    author={Minjie Wang and Da Zheng and Zihao Ye and Quan Gan and Mufei Li and Xiang Song and Jinjing Zhou and Chao Ma and Lingfan Yu and Yu Gai and Tianjun Xiao and Tong He and George Karypis and Jinyang Li and Zheng Zhang},
    year={2019},
    journal={arXiv preprint arXiv:1909.01315}
}

# TODO: Reference to paper:
https://arxiv.org/abs/1606.09375

conda install -c dglteam/label/th23_cu121 dgl

conda install -c pytorch torchdata

conda install pydantic -c conda-forge

# Common libs

In [ ]:
# noinspection PyUnresolvedReferences
from importlib import reload

# Load SEED Dataset

### Load RAW EEG

In [ ]:
import sys

from dataset_processing.seed_dataset_loader import SeedDatasetLoader

sampling_frequency = 200  # 200 Hz

_loader = SeedDatasetLoader(fs=sampling_frequency)

In [ ]:
labels = _loader.get_labels()
labels

In [ ]:
channel_order = _loader.get_channel_order()
channel_order

In [ ]:
_eeg_data_df = _loader.get_eeg_data_df()

In [ ]:
_loader.plot_random_eeg()

In [ ]:
del _loader

### Data Augmentation

In [ ]:
from dataset_processing.eeg_augmentation import EEGAugmentation

_augmentor = EEGAugmentation(_eeg_data_df)
_augmented_df = _augmentor.augment_data()
del _augmentor, _eeg_data_df

# Pre-Training

In [ ]:
from torch.utils.data import DataLoader
from dataset_processing.eeg_dataset import EEGDataset
from model.pre_training.do_pre_training import PreTraining

# From the paper
pretraining_batch_size = 256
pretraining_epochs = 1000

In [ ]:
# TODO: Add in bachelor thesis how `num_workers` was chosen with code from w_testing_values notebook

# Custom cleanup function, useful when using the dataloader too much,
# as it's bugged and needs manual cleaning (because of Jupyter Notebook)
def cleanup_data_loader(loader):
    # noinspection PyProtectedMember
    if loader._iterator is not None:
        # noinspection PyProtectedMember
        loader._iterator._shutdown_workers()

### Dataset Loader

In [ ]:
num_workers = 5

pretraining_data_loader = DataLoader(
    EEGDataset(_augmented_df),
    batch_size=pretraining_batch_size,
    shuffle=True,
    pin_memory=True,
    persistent_workers=True,

    num_workers=num_workers,
    prefetch_factor=2,  # Default: 2 for `num_workers` > 0  # TODO: Maybe set to 3-4
)
# del _augmented_df

### Do the pre-training

In [ ]:
import gc

print(f"Garbage collector: collected {gc.collect()} objects.")

In [ ]:
# TODO: Use for simplified training: https://pytorch-ignite.ai/tutorials/beginner/01-getting-started/

In [ ]:
print(f"Starting pre-training with {num_workers} workers loading the dataset")

try:
    pretraining_model_trainer = PreTraining(
        data_loader=pretraining_data_loader,
        sampling_frequency=sampling_frequency,
        pretraining_model_save_dir="model_params/pretraining",
        scheduler_patience=50,
        early_stopping_patience=100,
        # epochs=pretraining_epochs,
        epochs=2000,
    )
    pretraining_model_trainer.train()
except Exception as e:
    print(e, file=sys.stderr)

# cleanup_data_loader(pretraining_data_loader)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs

# Fine-Tuning

In [ ]:
from torch.utils.data import DataLoader
from dataset_processing.eeg_dataset import EEGDataset
from model.fine_tuning.do_fine_tuning import FineTuning
import pandas as pd
from sklearn.model_selection import train_test_split

# From the paper
finetuning_batch_size = 128
finetuning_epochs = 20

### Dataset Loader

In [ ]:
num_workers = 5

# Customizable percentages
train_percentage = 0.7
eval_percentage = 0.3

# "Verdict" is the column name representing the verdict class
train_df_list = []
eval_df_list = []

# Ensure balanced datasets by splitting for each verdict class
for verdict in _augmented_df["Verdict"].unique():
    verdict_df = _augmented_df[_augmented_df["Verdict"] == verdict]
    train_df, eval_df = train_test_split(
        verdict_df,
        train_size=train_percentage,
        random_state=42,
        stratify=verdict_df["Verdict"]
    )
    train_df_list.append(train_df)
    eval_df_list.append(eval_df)

    # Print the sizes for each verdict
    print(f"Verdict: {verdict}")
    print(f"TOTAL size: {len(train_df) + len(eval_df)}")
    print(f"- Train size: {len(train_df)}")
    print(f"- Eval  size: {len(eval_df)}\n")

# Combine the balanced splits back into training and evaluation datasets
train_df = pd.concat(train_df_list).reset_index(drop=True)
eval_df = pd.concat(eval_df_list).reset_index(drop=True)

# Create DataLoaders
finetuning_data_loader = DataLoader(
    EEGDataset(train_df),
    batch_size=finetuning_batch_size,
    shuffle=True,
    pin_memory=True,
    persistent_workers=True,
    num_workers=num_workers,
    prefetch_factor=2,
)

finetuning_data_loader_eval = DataLoader(
    EEGDataset(eval_df),
    batch_size=finetuning_batch_size,
    shuffle=False,
    pin_memory=True,
    persistent_workers=True,
    num_workers=num_workers,
    prefetch_factor=2,
)

### Do the fine-tuning

In [ ]:
pretraining_model = PreTraining(
    data_loader=None,
    sampling_frequency=sampling_frequency,
    pretraining_model_save_dir="model_params/pretraining",
    scheduler_patience=50,
    early_stopping_patience=100,
    epochs=2000,
    to_train=False,
)
pretraining_model.load_model(2000)

In [ ]:
# torch.autograd.set_detect_anomaly(True)

try:
    finetuning = FineTuning(
        data_loader=finetuning_data_loader,
        data_loader_eval=finetuning_data_loader_eval,
        sampling_frequency=sampling_frequency,
        num_classes=3,

        ET=pretraining_model.ET,
        EF=pretraining_model.EF,
        PT=pretraining_model.PT,
        PF=pretraining_model.PF,

        finetuning_model_save_dir="model_params/finetuning",
        # epochs=finetuning_epochs,
        epochs=20,
    )
    finetuning.train()
except Exception as e:
    print(e, file=sys.stderr)

In [ ]:
finetuning_model = FineTuning(
    data_loader=finetuning_data_loader,
    data_loader_eval=finetuning_data_loader_eval,
    sampling_frequency=sampling_frequency,
    num_classes=3,

    ET=pretraining_model.ET,
    EF=pretraining_model.EF,
    PT=pretraining_model.PT,
    PF=pretraining_model.PF,

    finetuning_model_save_dir="model_params/finetuning",
    to_train=False,
)
finetuning_model.load_model(20)

eval_accuracy, avg_eval_loss = finetuning_model.do_eval_epoch()
print(
    f"Eval Loss: {avg_eval_loss:.4f}, "
    f"Eval Accuracy: {eval_accuracy:.4f}"
)

# Ideas

Do a correlation matrix between the channels of the EEG signals.
Then when doing the joint whatever model, use the "distances" between the channels (like the hamming distance but not really), as a "weight" for training the joining etc.

Or maybe just output something that could show each channel's contribution towards the final emotion prediction.